In [10]:
DJIA_STOCKS = ['MSFT', 'AAPL', 'AMZN', 'JPM', 'WMT', 'V', 'UNH', 'PG', 'JNJ', 'HD', 'MRK', 'CVX', 'KO', 'CRM', 'CSCO', 'MCD', 'DIS', 'VZ', 'AXP', 'AMGN', 'CAT', 'IBM', 'GS', 'HON', 'INTC', 'NIKE', 'BA', 'MMM', 'TRV', 'DOW']
len(DJIA_STOCKS) == 30
# No pruning of data needed.

True